In [1]:
import math

import pandas as pd

In [2]:
points = pd.read_excel('DataSet.xlsx', sheet_name='Входные данные для анализа', usecols=range(7))

In [3]:
points.head(10)

,№ точки,"Адрес точки, г. Краснодар",Когда подключена точка?,Карты и материалы доставлены?,Кол-во дней после выдачи последней карты,Кол-во одобренных заявок,Кол-во выданных карт
0,1,"ул. Ставропольская, д. 140",вчера,нет,0.0,0.0,0.0
1,2,"ул. им. Максима Горького, д. 128",давно,да,3.0,15.0,3.0
2,3,"ул. им. Дзержинского, д. 100",давно,да,3.0,9.0,1.0
3,4,"ул. Красноармейская, д. 126",давно,да,0.0,38.0,23.0
4,5,"х. Ленина, п/о. 37",давно,нет,0.0,14.0,0.0
5,6,"тер. Пашковский жилой массив, ул. Крылатая, д. 2",давно,да,12.0,19.0,1.0
6,7,"ул. Восточно-Кругликовская, д. 64/2",давно,да,27.0,19.0,12.0
7,8,"ул. Красных Партизан, д. 439",давно,да,33.0,84.0,63.0
8,9,"ул. Таманская, д. 153 к. 3, кв. 2",давно,да,2.0,15.0,1.0
9,10,"ул. им. Дзержинского, д. 165",давно,да,0.0,19.0,0.0


In [4]:
points.dropna(inplace=True)

**Задача 1**: Выезд на точку для стимулирования выдач | Высокий | 4 часа
**Задача 2**: Обучение агента	| Средний | 2 часа
**Задача 3**: Доставка карт и материалов | Низкий | 1,5 часа


In [5]:
def set_task(point):
    if point["Когда подключена точка?"] == "вчера" or point["Карты и материалы доставлены?"] == "нет":
        return 3
    elif point['Кол-во выданных карт'] / point['Кол-во одобренных заявок'] <= 0.5:
        return 2
    elif (point["Кол-во дней после выдачи последней карты"] >= 7 and point['Кол-во одобренных заявок'] >= 1) or point["Кол-во дней после выдачи последней карты"] >= 14:
        return 1
    else:
        return 0

In [6]:
points["№ Задачи"] = points.apply(set_task, axis=1)

In [7]:
points.sort_values(by='№ Задачи', inplace=True)

In [8]:
points.head(10)

,№ точки,"Адрес точки, г. Краснодар",Когда подключена точка?,Карты и материалы доставлены?,Кол-во дней после выдачи последней карты,Кол-во одобренных заявок,Кол-во выданных карт,№ Задачи
3,4,"ул. Красноармейская, д. 126",давно,да,0.0,38.0,23.0,0
26,27,"ул. Уральская, д. 166/3",давно,да,3.0,43.0,29.0,0
33,34,"ул. Красная, д. 176",давно,да,76.0,82.0,72.0,1
34,35,"ул. Уральская, д. 79/1",давно,да,23.0,32.0,21.0,1
10,11,"ст-ца. Елизаветинская, ул. Широкая, д. 260",давно,да,15.0,29.0,15.0,1
7,8,"ул. Красных Партизан, д. 439",давно,да,33.0,84.0,63.0,1
36,37,"ул. Красная, д. 149",давно,да,9.0,10.0,7.0,1
29,30,"ул. Коммунаров, д. 258",давно,да,16.0,45.0,30.0,1
6,7,"ул. Восточно-Кругликовская, д. 64/2",давно,да,27.0,19.0,12.0,1
28,29,"ул. им. Кирилла Россинского, д. 61/1",давно,да,6.0,19.0,5.0,2


In [9]:
points["№ Задачи"].value_counts()

№ Задачи
2    21
3    11
1     7
0     2
Name: count, dtype: int64

In [10]:
# from geopy.geocoders import Nominatim
import requests
from yandex_geocoder import Client


client = Client("22900526-289d-40ca-8382-6c3a2282e733")
token = '5b3ce3597851110001cf6248c6536c473ace4987b80c93db6693db1c'

def matrix(address: list[str, str], profile=0):
    coord_1 = client.coordinates(address[0])
    coord_2 = client.coordinates(address[1])

    locations = [[float(coord_1[1]), float(coord_1[0])], [float(coord_2[1]), float(coord_2[0])]]

    headers = {
        'Content-Type': 'application/json; charset=utf-8',
        'Accept': 'application/json',
        'Authorization': token
    }
    profile_dict = {
        0: 'driving-car',
        1: 'foot-walking'
    }
    data = {"locations":[i[::-1] for i in locations],"metrics":["distance","duration"],"units":"m"}
    res = requests.post(f'https://api.openrouteservice.org/v2/matrix/{profile_dict[profile]}',
                        headers=headers,
                        json=data).json()
    ps = dict(durations = res['durations'][0][1], distances = res['distances'][0][1])

    return math.ceil(ps["durations"] / 60)

In [11]:
result = matrix(['г. Краснодар, ул. Красноармейская, д. 126', 'г. Краснодар, ул. Уральская, д. 166/3'])
print(result)

16


In [12]:
employees = pd.read_excel('DataSet.xlsx', sheet_name='Справочник сотрудников')

In [13]:
centres = employees["Адрес локации"].unique()

In [14]:
def set_grade(employee_grade):
    match employee_grade:
        case "Синьор":
            return [1,2,3]
        case "Мидл":
            return [2,3]
        case "Джун":
            return [3]

In [15]:
employees["Решаемые задачи"] = employees["Грейд"].apply(set_grade)

In [16]:
employees

,ФИО,Адрес локации,Грейд,Решаемые задачи
0,Дерягин Никита Владимирович,"Краснодар, Красная, д. 139",Синьор,"[1, 2, 3]"
1,Петрошев Валерий Павлович,"Краснодар, Красная, д. 139",Мидл,"[2, 3]"
2,Евдокимов Давид Тихонович,"Краснодар, Красная, д. 139",Джун,[3]
3,Андреев Гордий Данилович,"Краснодар, В.Н. Мачуги, 41",Синьор,"[1, 2, 3]"
4,Иванов Адам Федорович,"Краснодар, В.Н. Мачуги, 41",Мидл,"[2, 3]"
5,Бобылёв Ипполит Альбертович,"Краснодар, В.Н. Мачуги, 41",Джун,[3]
6,Беляева Евгения Антоновна,"Краснодар, Красных Партизан, 321",Мидл,"[2, 3]"
7,Николаев Азарий Платонович,"Краснодар, Красных Партизан, 321",Джун,[3]


In [18]:
employees_in_task = employees.copy()
employees_in_task["Кол-во отработанных минут"] = 0
employees_in_task["Номера взятых задач"] = employees_in_task["Кол-во отработанных минут"].apply(lambda x: [])

In [19]:
points_tomorrow = {
    '№ точки': [],
    'Адрес точки, г. Краснодар': [],
    'Когда подключена точка?': [],
    'Карты и материалы доставлены?': [],
    'Кол-во дней после выдачи последней карты': [],
    'Кол-во одобренных заявок': [],
    'Кол-во выданных карт': [],
    '№ Задачи': []
}

points_tomorrow = pd.DataFrame(points_tomorrow)

In [21]:
employees_in_task

,ФИО,Адрес локации,Грейд,Решаемые задачи,Кол-во отработанных минут,Номера взятых задач
0,Дерягин Никита Владимирович,"Краснодар, Красная, д. 139",Синьор,"[1, 2, 3]",0,[]
1,Петрошев Валерий Павлович,"Краснодар, Красная, д. 139",Мидл,"[2, 3]",0,[]
2,Евдокимов Давид Тихонович,"Краснодар, Красная, д. 139",Джун,[3],0,[]
3,Андреев Гордий Данилович,"Краснодар, В.Н. Мачуги, 41",Синьор,"[1, 2, 3]",0,[]
4,Иванов Адам Федорович,"Краснодар, В.Н. Мачуги, 41",Мидл,"[2, 3]",0,[]
5,Бобылёв Ипполит Альбертович,"Краснодар, В.Н. Мачуги, 41",Джун,[3],0,[]
6,Беляева Евгения Антоновна,"Краснодар, Красных Партизан, 321",Мидл,"[2, 3]",0,[]
7,Николаев Азарий Платонович,"Краснодар, Красных Партизан, 321",Джун,[3],0,[]


In [23]:
for index, point in points.iterrows():
    task_number = point['№ Задачи']
    if task_number == 0:
        continue
    elif task_number == 1:
        task_time = 240
    elif task_number == 2:
        task_time = 120
    elif task_number == 3:
        task_time = 90
    else:
        continue

    best_travel_time = 10000
    best_employee_id = None
    # task_accepted = False
    origin = 'Краснодар, ' + point['Адрес точки, г. Краснодар']

    suitable_employees = employees_in_task[employees_in_task['Решаемые задачи'].apply(lambda x: task_number in x)]
    suitable_employees = suitable_employees.sort_values(by=['Грейд'])

    if task_number == 3:
        junior = suitable_employees[suitable_employees["Грейд"] == "Джун"]
        if junior[junior['Кол-во отработанных минут'] >= 640].sum() < len(junior):
            suitable_employees = junior
        else:
            suitable_employees = suitable_employees[suitable_employees["Грейд"] != "Джун"]

    if task_number == 2:
        middle = suitable_employees[suitable_employees["Грейд"] == "Мидл"]
        if middle[middle['Кол-во отработанных минут'] >= 640].sum() < len(middle):
            suitable_employees = middle
        else:
            suitable_employees = suitable_employees[suitable_employees["Грейд"] == "Синьор"]

    for e_index, employee in suitable_employees.iterrows():

        if employee["Кол-во отработанных часов"] + task_time >= 640:
            continue

        destination = employee["Адрес локации"]
        travel_time = matrix([origin, destination])
        if travel_time < best_travel_time:
            best_travel_time = travel_time
            best_employee_id = e_index

    if best_employee_id is None:
        points_tomorrow = pd.concat([points_tomorrow, pd.DataFrame([point])])

    else:
        employees_in_task.loc[best_employee_id, "Кол-во отработанных минут"] += best_travel_time + task_time
        employees_in_task.loc[best_employee_id, "Номера взятых задач"].append(index)
        employees_in_task.loc[best_employee_id, "Адрес локации"] = origin


KeyError: 'Кол-во отработанных часов'

In [28]:
points_tomorrow

,№ точки,"Адрес точки, г. Краснодар",Когда подключена точка?,Карты и материалы доставлены?,Кол-во дней после выдачи последней карты,Кол-во одобренных заявок,Кол-во выданных карт,№ Задачи
0,2.0,"ул. им. Максима Горького, д. 128",давно,да,3.0,15.0,3.0,2.0
1,2.0,"ул. им. Максима Горького, д. 128",давно,да,3.0,15.0,3.0,2.0


In [35]:
employees_in_task[employees_in_task['Кол-во отработанных минут'] >= 640]["ФИО"].sum() < 3

True

In [30]:
employees

,ФИО,Адрес локации,Грейд,Решаемые задачи
0,Дерягин Никита Владимирович,"Краснодар, Красная, д. 139",Синьор,"[1, 2, 3]"
1,Петрошев Валерий Павлович,"Краснодар, Красная, д. 139",Мидл,"[2, 3]"
2,Евдокимов Давид Тихонович,"Краснодар, Красная, д. 139",Джун,[3]
3,Андреев Гордий Данилович,"Краснодар, В.Н. Мачуги, 41",Синьор,"[1, 2, 3]"
4,Иванов Адам Федорович,"Краснодар, В.Н. Мачуги, 41",Мидл,"[2, 3]"
5,Бобылёв Ипполит Альбертович,"Краснодар, В.Н. Мачуги, 41",Джун,[3]
6,Беляева Евгения Антоновна,"Краснодар, Красных Партизан, 321",Мидл,"[2, 3]"
7,Николаев Азарий Платонович,"Краснодар, Красных Партизан, 321",Джун,[3]


In [36]:
points_tomorrow = {
    key: [] for key in points.columns
}

In [37]:
points_tomorrow = pd.DataFrame(points_tomorrow)

In [38]:
points_tomorrow

,№ точки,"Адрес точки, г. Краснодар",Когда подключена точка?,Карты и материалы доставлены?,Кол-во дней после выдачи последней карты,Кол-во одобренных заявок,Кол-во выданных карт,№ Задачи
